In [1]:
import pandas as pd

In [3]:
masters = pd.read_csv("../data-version2/raw/master-programs/master-programs.csv")

In [5]:
masters.head()

,field,name,university,duration,url,language,city,country,mode,deadline,pace,tution_amount,tution_currency
0,Administration-Studies,M.S. in Aviation Management And Logistics,Kent State University - College of Aeronautics...,2 years,NaN,"[""English""]",Kent,USA,"[""Online""]",NaN,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration-Studies,Master of Science in Operations Management,Manderson Graduate School of Business The Univ...,NaN,NaN,[],Tuscaloosa,USA,"[""Campus""]",NaN,"[""Full-time""]",NaN,NaN
2,Administration-Studies,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,University of Argentinian Social Studies,NaN,NaN,"[""Spanish (Argentina)""]",545,Argentina,"[""Campus""]",NaN,[],NaN,NaN
3,Administration-Studies,"MS Supply Chain, Logistics and Innovations",EM Normandie Business School,15 months,NaN,"[""French""]",Le Havre,France,"[""Campus""]",NaN,"[""Full-time""]",12500.0,EUR
4,Administration-Studies,Master of Logistics Management (without thesis),Izmir University of Economics,2 years,NaN,"[""English""]",Izmir,Turkey,"[""Campus""]",NaN,"[""Full-time""]",8500.0,USD


In [4]:
masters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23367 entries, 0 to 23366
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   field            23367 non-null  object 
 1   name             23367 non-null  object 
 2   university       23367 non-null  object 
 3   duration         17166 non-null  object 
 4   url              2197 non-null   object 
 5   language         23367 non-null  object 
 6   city             23366 non-null  object 
 7   country          23361 non-null  object 
 8   mode             23367 non-null  object 
 9   deadline         2975 non-null   object 
 10  pace             23367 non-null  object 
 11  tution_amount    13204 non-null  float64
 12  tution_currency  13154 non-null  object 
dtypes: float64(1), object(12)
memory usage: 2.3+ MB


In [12]:
# masters program table will not contain university name, city and country because they will be in a separate table

column_names : dict = {
    
    "pace"  : "schedule",
    "field" : "field_of_study",
    
}

columns_to_drop = ["university", "city", "country"]

masters_clean = masters.drop(columns_to_drop, axis=1)
masters_clean.rename(column_names, axis=1, inplace=True)

universities = masters[["university", "city", "country"]]

In [14]:
masters_clean.head()

,field_of_study,name,duration,url,language,mode,deadline,schedule,tution_amount,tution_currency
0,Administration-Studies,M.S. in Aviation Management And Logistics,2 years,NaN,"[""English""]","[""Online""]",NaN,"[""Full-time"",""Part-time""]",42025.0,USD
1,Administration-Studies,Master of Science in Operations Management,NaN,NaN,[],"[""Campus""]",NaN,"[""Full-time""]",NaN,NaN
2,Administration-Studies,MASTER'S DEGREE IN COLLECTIVE LABOR BARGAINING,NaN,NaN,"[""Spanish (Argentina)""]","[""Campus""]",NaN,[],NaN,NaN
3,Administration-Studies,"MS Supply Chain, Logistics and Innovations",15 months,NaN,"[""French""]","[""Campus""]",NaN,"[""Full-time""]",12500.0,EUR
4,Administration-Studies,Master of Logistics Management (without thesis),2 years,NaN,"[""English""]","[""Campus""]",NaN,"[""Full-time""]",8500.0,USD


In [15]:
universities.head()

,university,city,country
0,Kent State University - College of Aeronautics...,Kent,USA
1,Manderson Graduate School of Business The Univ...,Tuscaloosa,USA
2,University of Argentinian Social Studies,545,Argentina
3,EM Normandie Business School,Le Havre,France
4,Izmir University of Economics,Izmir,Turkey


In [23]:
universities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23367 entries, 0 to 23366
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   university  23367 non-null  object
 1   city        23366 non-null  object
 2   country     23361 non-null  object
dtypes: object(3)
memory usage: 547.8+ KB


In [21]:
# we have duplicates in dataset need to remove them

university_clean = universities.drop_duplicates()

In [22]:
university_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4573 entries, 0 to 23359
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   university  4573 non-null   object
 1   city        4572 non-null   object
 2   country     4571 non-null   object
dtypes: object(3)
memory usage: 142.9+ KB


In [24]:
# we need to create user entity for each university
# we need email, password_hashed, role=Role.UNIVERSITY, registration_date=null 
